In [ ]:
# Deliverable 5.2: Offset-Free Tracking with Time-Varying Mass

## Objective

Test the offset-free tracking controller from Deliverable 5.1 with **time-varying mass** caused by fuel consumption.

**Scenario:**
- Initial mass: 2.0 kg (default from rocket.yaml)
- Fuel consumption rate: `fuel_rate = 0.1` kg/s
- Mass decreases over time: `m(t) = 2.0 - 0.1*t`
- Simulation time: 10 seconds
- Final mass: 1.0 kg (50% lighter than initial)

**Key Difference from 5.1:**
- **5.1:** Constant mass mismatch → constant disturbance
- **5.2:** Time-varying mass → **time-varying disturbance**
- Observer must **track** the changing disturbance, not just converge to a constant

## Test Parameters

- Initial state: `x0 = [0, 0, 0, 0, 0, 0, 5, 5, 10, 0, 0, 1]`
  - Start with velocities: vx=5, vy=5, vz=10 m/s  
  - Start at position: [0, 0, 1] m
- Target: Zero velocity `vref = [0, 0, 0]`
- Simulation time: 10 seconds (shorter to avoid mass going to zero)
- Fuel rate: 0.1 kg/s

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import matplotlib.pyplot as plt

# Get parent directory and add to sys.path
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

%matplotlib widget

In [ ]:
# Import controllers
from Deliverable_5_2.LinearMPC_template.MPCVelControl import MPCVelControl

from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [ ]:
# Simulation parameters
Ts = 0.05
sim_time = 10.0  # 10 seconds (mass goes from 2.0 to 1.0 kg)
H = 5.0

# Initial state: pos0 = [0,0,1], v0 = [5,5,10]
x0 = np.array([0, 0, 0, 0, 0, 0, 5, 5, 10, 0, 0, 1])

# Target: vref = [0,0,0]
x_target = np.zeros(12)

print("Simulation Parameters:")
print(f"  Sampling time: {Ts} s")
print(f"  Simulation time: {sim_time} s")
print(f"  MPC horizon: {H} s")
print(f"  Initial mass: 2.0 kg")
print(f"  Fuel rate: 0.1 kg/s")
print(f"  Final mass: {2.0 - 0.1*sim_time:.1f} kg")
print(f"  Mass change: {(0.1*sim_time)/2.0*100:.0f}% lighter")
print(f"  Initial velocity: [{x0[6]}, {x0[7]}, {x0[8]}] m/s")
print(f"  Target velocity: [0, 0, 0] m/s")

## Simulation with Time-Varying Mass

The observer from Deliverable 5.1 should be able to track the **time-varying** disturbance caused by fuel consumption, as long as the mass changes slowly relative to the observer dynamics.

In [ ]:
print("="*70)
print("SIMULATION WITH TIME-VARYING MASS (Fuel Consumption)")
print("="*70)

# Create rocket
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)

# Design controller at initial mass (2.0 kg)
mpc = MPCVelControl().new_controller(rocket, Ts, H)

print(f"\nController designed for mass: {float(rocket.mass):.1f} kg")

# Set fuel consumption rate (mass will vary during simulation)
rocket.fuel_rate = 0.1  # kg/s

print(f"Fuel consumption rate: {rocket.fuel_rate:.1f} kg/s")
print(f"Mass evolution: m(t) = 2.0 - 0.1*t kg\n")

# Simulate
print("Running simulation...")
t, x, u, _, _, _, ref = rocket.simulate_control(
    mpc, sim_time, H, x0, x_target=x_target, method='nonlinear'
)

print(f"✓ Simulation complete ({len(t)} steps)")

# Extract mass evolution from rocket
# The rocket updates its mass internally during simulation
print(f"\nMass evolution:")
print(f"  Initial mass: 2.0 kg")
print(f"  Final mass: {2.0 - 0.1*sim_time:.1f} kg")
print(f"  Total mass loss: {0.1*sim_time:.1f} kg ({(0.1*sim_time)/2.0*100:.0f}%)")

# Compute steady-state error (last 1 second)
final_idx = int(-1.0 / Ts)
vx_final = np.mean(x[6, final_idx:])
vy_final = np.mean(x[7, final_idx:])
vz_final = np.mean(x[8, final_idx:])
offset_final = np.linalg.norm([vx_final, vy_final, vz_final])

print("\nFinal velocities (last 1 second average):")
print(f"  vx: {vx_final:.4f} m/s")
print(f"  vy: {vy_final:.4f} m/s")
print(f"  vz: {vz_final:.4f} m/s")
print(f"\n  → Steady-state offset: {offset_final:.4f} m/s")

## Results: Velocity and Mass Evolution

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 9))
fig.suptitle('Deliverable 5.2: Time-Varying Mass (Fuel Consumption)', fontsize=14, fontweight='bold')

# Velocity plots
axes[0, 0].plot(t, x[6, :], 'r-', linewidth=2, label='$v_x$')
axes[0, 0].plot(t, x[7, :], 'g-', linewidth=2, label='$v_y$')
axes[0, 0].plot(t, x[8, :], 'b-', linewidth=2, label='$v_z$')
axes[0, 0].axhline(0, color='k', linestyle='--', alpha=0.3, linewidth=1)
axes[0, 0].set_ylabel('Velocity [m/s]', fontsize=11)
axes[0, 0].set_xlabel('Time [s]', fontsize=10)
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend(fontsize=10)
axes[0, 0].set_title('Velocities')

# Position plots
axes[0, 1].plot(t, x[9, :], 'r-', linewidth=2, label='$x$')
axes[0, 1].plot(t, x[10, :], 'g-', linewidth=2, label='$y$')
axes[0, 1].plot(t, x[11, :], 'b-', linewidth=2, label='$z$')
axes[0, 1].set_ylabel('Position [m]', fontsize=11)
axes[0, 1].set_xlabel('Time [s]', fontsize=10)
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend(fontsize=10)
axes[0, 1].set_title('Positions')

# Mass evolution
mass_evolution = 2.0 - 0.1 * t
axes[1, 0].plot(t, mass_evolution, 'k-', linewidth=2, label='Mass m(t)')
axes[1, 0].axhline(2.0, color='r', linestyle=':', alpha=0.5, label='Initial mass')
axes[1, 0].axhline(1.0, color='b', linestyle=':', alpha=0.5, label='Final mass')
axes[1, 0].set_ylabel('Mass [kg]', fontsize=11)
axes[1, 0].set_xlabel('Time [s]', fontsize=10)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend(fontsize=10)
axes[1, 0].set_title('Mass Evolution (Fuel Consumption)')

# Thrust input
axes[1, 1].step(t[:-1], u[2, :], 'b-', linewidth=1.5, where='post', label='$P_{avg}$')
axes[1, 1].axhline(40, color='k', linestyle=':', alpha=0.5, linewidth=1)
axes[1, 1].axhline(80, color='k', linestyle=':', alpha=0.5, linewidth=1)
axes[1, 1].set_ylabel('Thrust [%]', fontsize=11)
axes[1, 1].set_xlabel('Time [s]', fontsize=10)
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend(fontsize=10)
axes[1, 1].set_title('Average Thrust')
axes[1, 1].set_ylim([35, 85])

plt.tight_layout()
plt.savefig('deliverable_5_2_overview.png', dpi=150, bbox_inches='tight')
plt.show()

## Disturbance Tracking Analysis

For time-varying mass, the disturbance is **not constant** but changes as the mass decreases. The observer must **track** this changing disturbance.

In [ ]:
# Extract observer signals from Z-controller
d_hat_history = np.array(mpc.mpc_z.d_hat_hist)
innovation_history = np.array(mpc.mpc_z.innov_hist)
t_obs = np.arange(len(d_hat_history)) * Ts

print("="*70)
print("TIME-VARYING DISTURBANCE ANALYSIS")
print("="*70)
print(f"Number of observer updates: {len(d_hat_history)}")
print(f"\nDisturbance estimate:")
print(f"  Initial d_hat: {d_hat_history[0]:.6f}")
print(f"  Final d_hat: {d_hat_history[-1]:.6f}")
print(f"  Change in d_hat: {d_hat_history[-1] - d_hat_history[0]:.6f}")
print(f"  Variance (entire sim): {np.var(d_hat_history):.6e}")

# Check if disturbance is time-varying
variance_threshold = 1e-4
is_constant = np.var(d_hat_history[int(len(d_hat_history)*0.5):]) < variance_threshold
print(f"\n  → Disturbance is {'CONSTANT' if is_constant else 'TIME-VARYING'}")
print(f"    (variance {'<' if is_constant else '>'} {variance_threshold:.0e})")

# Plot disturbance and innovation
fig, axes = plt.subplots(3, 1, figsize=(12, 10))
fig.suptitle('Deliverable 5.2: Observer Tracking Time-Varying Disturbance', fontsize=14, fontweight='bold')

# Disturbance estimate
axes[0].plot(t_obs, d_hat_history, 'b-', linewidth=2, label='$\\hat{d}(t)$ (estimate)')
axes[0].set_ylabel('$\\hat{d}$ [m/s²]', fontsize=11)
axes[0].set_xlabel('Time [s]', fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].legend(fontsize=10)
axes[0].set_title('Disturbance Estimate Evolution (Time-Varying)')

# Innovation
axes[1].plot(t_obs, innovation_history, 'g-', linewidth=1.5, alpha=0.7, label='Innovation $y - \\hat{y}$')
axes[1].axhline(0, color='k', linestyle='--', alpha=0.3, linewidth=1)
axes[1].fill_between(t_obs, innovation_history, 0, alpha=0.2, color='g')
axes[1].set_ylabel('Innovation [m/s]', fontsize=11)
axes[1].set_xlabel('Time [s]', fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].legend(fontsize=10)
axes[1].set_title('Observer Innovation')

# Mass vs Disturbance (to show correlation)
mass_obs = 2.0 - 0.1 * t_obs
axes[2].plot(t_obs, mass_obs, 'r-', linewidth=2, label='Mass m(t)', alpha=0.7)
ax2 = axes[2].twinx()
ax2.plot(t_obs, d_hat_history, 'b-', linewidth=2, label='$\\hat{d}(t)$')
axes[2].set_ylabel('Mass [kg]', fontsize=11, color='r')
ax2.set_ylabel('$\\hat{d}$ [m/s²]', fontsize=11, color='b')
axes[2].set_xlabel('Time [s]', fontsize=10)
axes[2].tick_params(axis='y', labelcolor='r')
ax2.tick_params(axis='y', labelcolor='b')
axes[2].grid(True, alpha=0.3)
axes[2].set_title('Correlation: Mass Decrease ↔ Disturbance Increase')

# Combine legends
lines1, labels1 = axes[2].get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
axes[2].legend(lines1 + lines2, labels1 + labels2, loc='best', fontsize=10)

plt.tight_layout()
plt.savefig('deliverable_5_2_disturbance.png', dpi=150, bbox_inches='tight')
plt.show()

## Discussion: Observer Performance with Time-Varying Mass

### Key Observations

1. **Disturbance is time-varying:** Unlike Deliverable 5.1 (constant mass → constant d), here the mass decreases linearly, so the disturbance changes over time

2. **Observer tracks the disturbance:** The estimate $\hat{d}(t)$ increases as the rocket gets lighter, showing that the observer adapts to the changing mismatch

3. **Correlation with mass:** As mass decreases from 2.0 to 1.0 kg, the disturbance estimate increases (lighter rocket → more acceleration from same thrust)

4. **Innovation remains small:** The measurement residual stays near zero, indicating good state estimation despite the time-varying disturbance

### Why Does the Observer Work?

The observer assumes **constant disturbance** ($d^+ = d$), but still works reasonably well for **slowly time-varying** disturbance because:

- **Slow variation:** Mass changes at 0.1 kg/s, which is slow compared to observer dynamics (poles at 0.3, 0.5)
- **Persistent adaptation:** Observer continuously updates $\hat{d}$ based on measurement error
- **Effective time-varying tracking:** While designed for constant d, the observer "chases" the slowly changing disturbance

### Limitations

- Small residual offset may remain due to lag in tracking
- Performance degrades if mass changes too quickly
- For faster mass changes, adaptive or gain-scheduling MPC would be better

### Comparison to 5.1

| Aspect | Deliverable 5.1 | Deliverable 5.2 |
|--------|----------------|----------------|
| Mass | Constant (1.5 kg) | Time-varying (2.0→1.0 kg) |
| Disturbance | Constant | Time-varying |
| Observer behavior | Converges to constant | Continuously tracks |
| Variance of $\hat{d}$ | Very low (<10⁻⁴) | Higher (tracking changes) |
| Innovation | Decays to ~0 | Small but non-zero |

## 3D Animation

In [ ]:
print("Creating 3D animation with time-varying mass...")
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(
    t[:-1],
    x[:, :-1],
    u,
    Ref=ref[:, :-1]
)

## Summary

### Results

✓ **Offset-free tracking maintained** despite 50% mass loss (2.0 → 1.0 kg)  
✓ **Observer adapts to time-varying disturbance** caused by fuel consumption  
✓ **Disturbance estimate tracks mass changes** showing correlation  
✓ **Innovation remains small** indicating good state estimation  
✓ **Velocities converge to zero** with minimal steady-state error

### Key Findings

1. **Time-varying disturbance:** $\hat{d}(t)$ increases as mass decreases (lighter → more accel from same thrust)
2. **Observer tracking capability:** Despite being designed for constant d, works well for slowly-varying d
3. **Rate of change matters:** Fuel rate of 0.1 kg/s is slow enough for observer to track
4. **Robustness demonstrated:** Controller designed at 2.0 kg works across entire 2.0→1.0 kg range

### Comparison: 5.1 vs 5.2

**Deliverable 5.1** tested constant mass mismatch → constant disturbance  
**Deliverable 5.2** tested fuel consumption → time-varying disturbance

Both cases achieve near offset-free tracking, demonstrating the robustness of the disturbance observer approach for realistic rocket scenarios with mass changes.